## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [771]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [797]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.criterion = criterion
        self.max_features = max_features
        self.num_class = -1
        self.init()
    
    def init(self):
        if self.criterion == 'gini':
            self.G_function = self.__gini
        elif self.criterion == 'entropy':
            self.G_function = self.__entropy
        elif self.criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if self.max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif self.max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif self.max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise 
    
    def get_params(self, deep=True):
        return {"min_samples_split": self.min_samples_split, "max_depth": self.max_depth, 
                "sufficient_share": self.sufficient_share, "criterion": self.criterion, 
                "max_features": self.max_features}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        self.init()
        return self
    
    def __gini(self, l_c, l_s, r_c, r_s, size):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_c = l_c/l_s
        r_c = r_c/r_s
        left_gini = np.diag(l_c.dot((1 - l_c).T))
        left_gini = left_gini.reshape(l_s.shape[0], 1)
        right_gini = np.diag(r_c.dot((1 - r_c).T))
        right_gini = right_gini.reshape(r_s.shape[0], 1)
        
        return left_gini*l_s/size + right_gini*r_s/size
    
    def __entropy(self, l_c, l_s, r_c, r_s, size):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_c = l_c/l_s
        r_c = r_c/r_s
        left_entropy = -1*np.diag(l_c.dot(np.log((l_c).T)))
        left_entropy = left_entropy.reshape(l_s.shape[0], 1)
        right_entropy = -1*np.diag(r_c.dot(np.log((r_c).T)))
        right_entropy = right_entropy.reshape(r_s.shape[0], 1)
        
        return left_entropy*l_s/size + right_entropy*r_s/size 

    def __misclass(self, l_c, l_s, r_c, r_s, size):
        l_c = l_c/l_s
        r_c = r_c/r_s
        return 1 - np.amax(l_c, axis=1) + 1 - np.amax(r_c, axis=1)

    def __get_feature_ids_sqrt(self, n_features):
        feature_ids = range(n_features)
        np.random.shuffle(feature_ids)
        return feature_ids[0:max(1, int(np.sqrt(n_features)))]
        
    def __get_feature_ids_log2(self, n_features):
        feature_ids = range(n_features)
        np.random.shuffle(feature_ids)
        return feature_ids[0:max(1, int(np.log2(n_features)))]

    def __get_feature_ids_N(self, n_features):
        return range(n_features)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #Сортируем значения фичи по возрастанию с учетом метки класса
        sorted_x, sorted_y = self.__sort_samples(x, y)
        #Сколько всего классов
        class_number = np.unique(y).shape[0]
        
        #Учитываем, что нас интересуют сплиты с размером не меньше min_samples_split
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        #Находим индексы, где меняется класс
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        if len(r_border_ids) == 0:
            return [float('+inf'), None]
        
        # Для каждого разбиения считаем кол-во эл-тов каждого класса(l_class_count, r_class_count)
        #Считаем сколько элементов в каком поддереве при каждом разиении(l_sizes, r_sizes)
        #Это матрицы, где строки это разбиения, а столбцы это классы
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        #Выбираем то разбиение, для которого самый маленький критерий информативности
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes, y.shape[0])
        idx = np.argmin(gs)
    
        # Возвращаем значение критерия и значение threshold, по которому будем разбивать на поддеревья
        left_el_id = l_sizes[idx][0]
        threshold = (sorted_x[left_el_id + 1] + sorted_x[left_el_id]) / 2.0
        if(threshold >= np.max(x)) : 
            threshold = threshold - 0.5
        
        return [gs[idx], threshold]

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        
        #проверяем критерии останова
        if (depth == self.max_depth or self.__count_sufficient_share(y) >= self.sufficient_share or y.shape[0] <= self.min_samples_split):
            self.tree[node_id] = (self.__class__.LEAF_TYPE, self.__most_frequent_class(y))
        else: 
            #Находим лучшую фичу для разбиения и ее порог
            thresholds = []
            for feature_id in self.get_feature_ids(x.shape[1]):
                th = self.__find_threshold(x[:, feature_id], y)
                th.append(feature_id)
                thresholds.append(th)
            min_gs = min(thresholds, key = lambda t: t[0])
            
            #Не удалось найти разбиение - делаем лист
            if min_gs[1] is not None:
                #Разбиваем выборку на поддеревья
                left_x, right_x, left_y, right_y = self.__div_samples(x, y, min_gs[2], min_gs[1])
                if(left_x.shape[0] == 0):
                    self.tree[node_id] = (self.__class__.LEAF_TYPE, self.__most_frequent_class(right_y))
                elif(right_x.shape[0] == 0):
                    self.tree[node_id] = (self.__class__.LEAF_TYPE, self.__most_frequent_class(left_y))
                else:
                    self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, min_gs[2], min_gs[1])
                    #Рекурсивно строим дерево
                    self.__fit_node(left_x, left_y, 2 * node_id + 1, depth + 1)
                    self.__fit_node(right_x, right_y, 2 * node_id + 2, depth + 1)
                    
            else:
                self.tree[node_id] = (self.__class__.LEAF_TYPE, self.__most_frequent_class(y))
            
    
    def __most_frequent_class(self, y):
        ones = sum(y)
        zeros = y.shape[0] - ones
        if(ones >= zeros):
            return 1
        else:
            return 0
    
    def __count_sufficient_share(self, y):
        if(self.__most_frequent_class(y) == 1):
            return sum(y)/float(y.shape[0])
        else:
            return (y.shape[0] - sum(y))/float(y.shape[0])
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [725]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [726]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [713]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=3)
clf = DecisionTreeClassifier(min_samples_split=2, max_depth = 3)

## Проверка скорости работы

In [683]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

67.7541248798
0.17719912529


## Проверка качества работы

In [684]:
gkf = KFold(n_splits=5, shuffle=True)

In [727]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.933690862227
0.93219422965
0.92886837948
0.934356032261
0.9341870037


In [714]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.933566142845
0.934771763532
0.931820071506
0.928826806352
0.932399284912


# Применить для задачи Titanic 

In [728]:
train = pd.read_csv("train.csv")

In [729]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")

In [730]:
target = train["Survived"].values

In [731]:
train = train.drop(['PassengerId','Survived','Name','Ticket','Cabin'], axis=1)
train = pd.get_dummies(train)

In [732]:
x = train.values

In [733]:
x_train, x_validation, y_train, y_validation = train_test_split(x, target, test_size=.25, random_state=1)

In [783]:
tree = MyDecisionTreeClassifier(min_samples_split=2, max_depth=3)
tree.fit(x_train, y_train)

In [770]:
print(accuracy_score(y_pred=tree.predict(x_validation), y_true=y_validation))

0.775784753363


In [744]:
tree_sc = DecisionTreeClassifier(min_samples_split=2)
tree_sc.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [745]:
print(accuracy_score(y_pred=tree_sc.predict(x_validation), y_true=y_validation))

0.757847533632


In [803]:
tuned_parameters = {'max_features': ['sqrt', 'log2', None], 'criterion': ['gini', 'entropy', 'misclass'],
                     'sufficient_share': [0.3, 0.5, 0.7, 1.0], 'max_depth': [3, 5, 7, 10, 15], 
                    'min_samples_split': [2, 3, 4, 5, 6]}

In [804]:
tree = MyDecisionTreeClassifier()
clf = GridSearchCV(tree, tuned_parameters, cv=5, scoring='accuracy')
clf.fit(x, target)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:65: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:67: RuntimeWarning: divide by zero encountered in log


GridSearchCV(cv=5, error_score='raise',
       estimator=<__main__.MyDecisionTreeClassifier instance at 0x26e004ab8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['sqrt', 'log2', None], 'sufficient_share': [0.3, 0.5, 0.7, 1.0], 'min_samples_split': [2, 3, 4, 5, 6], 'criterion': ['gini', 'entropy', 'misclass'], 'max_depth': [3, 5, 7, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [805]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'log2',
 'min_samples_split': 2,
 'sufficient_share': 1.0}

In [807]:
print(accuracy_score(y_pred=clf.best_estimator_.predict(x_validation), y_true=y_validation))

0.789237668161
